# Notebook **One** - Identifying the position(s) of Brightest Cluster Galaxies (BCGs)

## Import statements and reading history/sample

In [1]:
%matplotlib ipympl

from ident_run_setup import cosmo, side_length, HISTORY_FILE_PATH, load_history, proj_name, update_history, InteractiveView

from xga.products import Image

from astropy.wcs import WCS
from astropy.units import Quantity
from astropy.visualization import MinMaxInterval, LogStretch
import matplotlib
from matplotlib import pyplot as plt
# from matplotlib_scalebar.scalebar import ScaleBar
from matplotlib.colors import LinearSegmentedColormap
import numpy as np
import pandas as pd
pd.set_option('display.max_columns', 500)

%matplotlib ipympl

The framework I've designed for this process (i.e. these notebooks) is trying to keep a useful history of the process - it has recorded the images that we were able to download or generate, and which you'll be looking at in this notebook - and will also help you to more easily record the galaxies that you identify as being possible BCGs. 

**As this hasn't been tested much, it might be a good idea to keep your own backup copies of the coordinates you identify, by copying the 'history/bcg_ident_proj_save.json' file to some other place on your computer when you're done with each session of identifying BCGs - you could also change the name to add the date and time you made the copy** 

In [2]:
cur_history = load_history()

We read the total sample file into a Pandas dataframe:

In [3]:
# This reads the 'comma separated variable', or CSV/csv, file (which contains information about the sample of clusters
#  being investigated), from disk (on the hard-disk drive\HDD, or solid state disk\SSD) into memory (a much faster form of storage 
#  called Random Access Memory\RAM, where files being used by the operating system or currently running software live). We are using 
#  a Python module called 'Pandas' to read it in - think of it almost like a Python version of Excel
samp = pd.read_csv(cur_history['static_samp_file'])
# This method shows the top two lines of the loaded-in sample file
samp.head(2)

,name,cent_im_ra,cent_im_dec,redshift,r500,r500-,r500+,r2500,r2500-,r2500+,XCS_NAME,ang_prop_ratio
0,SDSSXCS-124,0.800578,-6.091818,0.247483,1181.028159,21.202221,23.202641,534.834740,7.579124,7.591855,XMMXCS J000312.1-060530.5,232.859967
1,SDSSXCS-2789,0.955540,2.068019,0.105285,1007.860978,17.194150,17.201505,438.706515,5.198301,5.213676,XMMXCS J000349.3+020404.8,115.802997


All cluster names:

In [4]:
print(list(samp['name'].values))

['SDSSXCS-124', 'SDSSXCS-2789', 'SDSSXCS-290', 'SDSSXCS-1018', 'SDSSXCS-134', 'SDSSXCS-119']


## A CAVEAT

For technical reasons that I won't explain here, the only to get this interactive plot process working in Jupyter Notebooks (in its current form at least) is to run the notebook for one cluster, then restart it and run it again for the next, and so on.

This stepping through of clusters will happen automatically - once the project history has an entry for BCG coordinates from your investigation, the next time the notebook is run it will move on to the next cluster without BCG coordinates. If you want to go back to a specific cluster and overwrite your first attempt, you can set the variable below to the cluster name (e.g. 'SDSSXCS-124', make sure the name is a string with the '' around it) - **before you run the notebook again, make sure to set `overwrite_cluster=None`, and note that None doesn't have any '' around it, as it is not a string**

In [5]:
# You can change this per the instructions above
overwrite_cluster = None

# Don't change this though
if overwrite_cluster is not None and overwrite_cluster not in samp['name'].values:
    raise ValueError("The overwriting cluster name you set does not correspond to a name in the sample, check "\
        "to make sure you typed it correctly!")

Here we identify the next cluster that doesn't have any recorded BCG identifications, which will be the next one you'll work on - this makes use of the 'history' that is being written as you go through the sample.

In [6]:
# Here we use the 'complete' flag stored in the BCG identification history to measure the number of clusters
#  for which the BCG candidate process is complete
# A bit of a silly step to just extract the 'ident_complete' info from the dictionaries of particular clusters, as they
#  could have other entries (like BCG info for instance)
just_complete = {n: {'ident_complete': cur_history['bcg_identification'][n]['ident_complete']} 
                 for n in cur_history['bcg_identification']}
ident_comp_df = pd.DataFrame(just_complete).T
num_bcg_comp = ident_comp_df['ident_complete'].sum()

# Show a quick progress update on identifying the BCGs
print('BCG candidate identification is {p}% complete.'.format(p=round((num_bcg_comp/cur_history['num_clusters'])*100, 2)))

if overwrite_cluster is None:
    # Finds the first cluster with a 'False' value in the 'ident_complete' column - though only if you haven't
    #  set an overwriting cluster name above
    cur_name = ident_comp_df.loc[~ident_comp_df['ident_complete']].index[0]
else:
    cur_name = overwrite_cluster

# Now we read out the relevant row of the sample table
cur_row = samp[samp['name'] == cur_name].iloc[0]
cur_row

BCG candidate identification is 0.0% complete.


name                            SDSSXCS-124
cent_im_ra                         0.800578
cent_im_dec                       -6.091818
redshift                           0.247483
r500                            1181.028159
r500-                             21.202221
r500+                             23.202641
r2500                             534.83474
r2500-                             7.579124
r2500+                             7.591855
XCS_NAME          XMMXCS J000312.1-060530.5
ang_prop_ratio                   232.859967
Name: 0, dtype: object

## Searching for BCG candidates

Optical and near-infrared wavelengths are where galaxies in general emit the most light, and thus are easiest to see and identify. You will be looking at optical images that I have taken from the 'Dark Energy Spectroscopic Instrument' (DESI) Legacy Survey, which is essentially  a set of uniformly processed observations (some were in the telescope archives from other projects, some they proposed for specifically) that they combined and analysed - this was all to identify targets for DESI (which takes optical/near-infrared *spectra* rather than the images you are looking at). They also have an [online viewer for their survey](https://www.legacysurvey.org/viewer#IC%202455) which is pretty great (if a little slow sometimes).

For this task you will also be able to view the X-ray emission from the galaxy cluster itself - this is in another image to the right hand side, and is mostly there to provide some context for where the galaxy cluster is and where to start looking for the BCGs. The X-ray data are from a European Space Agency (ESA) telescope called XMM-Newton, which is particularly good for a lot of galaxy cluster work.

### Loading/setting up images

#### DESI Legacy Survey (optical/near-infrared)

In [7]:
desi_ls_pth = cur_history['data_operations'][cur_name]['raw_images']['desi-ls']['im_path']
desi_ls_data = np.flip(plt.imread(desi_ls_pth), axis=0)

ls_wcs = WCS(naxis=2)
ls_cen = cur_history['data_operations'][cur_name]['raw_images']['desi-ls']['cen_pos']

ls_pix_scale = cur_history['data_operations'][cur_name]['raw_images']['desi-ls']['arcsec_per_pix']
ls_wcs.wcs.cdelt = [-1*(ls_pix_scale/3600), ls_pix_scale/3600]
ls_wcs.wcs.crpix = [desi_ls_data.shape[0]/2, desi_ls_data.shape[1]/2]
ls_wcs.wcs.crval = [ls_cen[0], ls_cen[1]]
ls_wcs.wcs.ctype = ['RA---TAN', 'DEC--TAN']

#### XMM (X-ray)

In [8]:
# xmm_pth = cur_history['data_operations'][cur_name]['raw_images']['desi-ls']['im_path']
# xmm_im = Image(xmm_pth, 'combined', 'combined', '', '', '', Quantity(0.5, 'keV'), Quantity(2.0, 'keV'))

xm_pth = "raw_images/{n}/xmm/{n}_0.5-2.0keV_comb.fits".format(n=cur_name)
xmm_im = Image(xm_pth, 'combined', 'combined', '', '', '', Quantity(0.5, 'keV'), Quantity(2.0, 'keV'))

FileNotFoundError: FITSIO read_header cannot open None, possibly because there is a problem with the file, it doesn't exist, or maybe an SFTP problem? This product is associated with None.

#### Putting the data together

In [ ]:
im_data = {'desi-ls': desi_ls_data,
           'xmm': xmm_im.data}

im_wcs = {'desi-ls': ls_wcs,
          'xmm': xmm_im.radec_wcs}

#### Controlling how the images are displayed

In [ ]:
im_scale = {'xmm': {}}
im_scale['xmm']['interval'] = MinMaxInterval()
im_scale['xmm']['stretch'] = LogStretch()
im_scale['xmm']['cmap'] = 'gnuplot2'

### The identification tool

**This is where you have to do your task**

In [ ]:
cur_im = InteractiveView(im_data, im_wcs, 'desi-ls', cur_name, im_scale=im_scale, figsize=(10, 5))